In [9]:
import pyspark

sc = pyspark.SparkContext()
# lines = sc.textFile(sys.argv[1])

RuntimeError: Java gateway process exited before sending its port number

In [10]:
sc

''

In [6]:
lines = sc.textFile('samples_output-20_06-20_08-1')
print(lines)

NameError: name 'sc' is not defined

In [2]:
import pandas as pd
def write_parquet_file():
    df = pd.read_csv('data/us_presidents.csv')
    df.to_parquet('tmp/us_presidents.parquet')
write_parquet_file()

SyntaxError: invalid syntax (<ipython-input-2-19d7feec9187>, line 2)